# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker


### Choosing the algorithm

Before save in S3, I decided to do a research about the best model to this kind of data, and because of the deadline I used pycatet(https://pycaret.org/) as auxiliar tool.
It proposed to me the ExtraTreesClassifier and it will be the algorithm that I will use.


In [2]:
import pycaret
from pycaret.classification import *                      
from pycaret.utils import enable_colab                

data_dir = './plagiarism_data/'

train = pd.read_csv(data_dir+'train.csv')

classi = setup(data = train,           
              target = '0')

compare_models()



,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Extra Trees Classifier,0.9800,0.9833,0.9667,1.0000,0.9800,0.9615,0.9667,0.0816
1,Decision Tree Classifier,0.9600,0.9667,0.9333,1.0000,0.9600,0.9231,0.9333,0.0021
2,Random Forest Classifier,0.9600,0.9833,0.9333,1.0000,0.9600,0.9231,0.9333,0.0152
3,CatBoost Classifier,0.9600,0.9667,0.9333,1.0000,0.9600,0.9231,0.9333,0.9731
4,Naive Bayes,0.9550,1.0000,0.9667,0.9750,0.9657,0.8615,0.8667,0.0018
5,Logistic Regression,0.9400,1.0000,0.9333,0.9750,0.9457,0.8776,0.8946,0.0038
6,K Neighbors Classifier,0.9400,0.9750,0.9000,1.0000,0.9400,0.8846,0.9000,0.0017
7,Extreme Gradient Boosting,0.9400,0.9667,0.9333,0.9750,0.9457,0.8776,0.8946,0.0054
8,Quadratic Discriminant Analysis,0.9350,0.9833,0.9333,0.9750,0.9457,0.8231,0.8333,0.0018
9,Light Gradient Boosting Machine,0.9350,0.9833,0.9667,0.9500,0.9514,0.8161,0.8279,0.0057


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=7694, verbose=0,
                     warm_start=False)

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [4]:
# should be the name of directory you created to save your features data
data_dir = './plagiarism_data/'

# set prefix, a descriptive name for a directory  
prefix = "plagiarism_data"

# upload all data to S3
input_data = sagemaker_session.upload_data(data_dir, key_prefix=prefix)

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

plagiarism_data/test.csv
plagiarism_data/train.csv
Test passed!


---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [6]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
os.system('pip install joblib')
import joblib
import pandas as pd
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier

## TODO: Import any additional libraries you need to define a model


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


## TODO: Complete the main code
if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a train

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [7]:
from sagemaker.sklearn.estimator import SKLearn
# your import and estimator code, here

FRAMEWORK_VERSION = "0.23-1"
output_path = 's3://{}/{}'.format(bucket, prefix)

estimator = SKLearn(source_dir ='source_sklearn',
        entry_point ='train.py',        
        role=role,
        train_instance_count=1,
        train_instance_type="ml.m5.xlarge",
        sagemaker_session=sagemaker_session,
        framework_version=FRAMEWORK_VERSION,
        output_path=output_path)

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [8]:
%%time

# Train your estimator on S3 training data

estimator.fit({'train': input_data})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-09-28 00:24:47 Starting - Starting the training job...
2020-09-28 00:24:49 Starting - Launching requested ML instances......
2020-09-28 00:26:02 Starting - Preparing the instances for training...
2020-09-28 00:26:37 Downloading - Downloading input data...
2020-09-28 00:27:10 Training - Downloading the training image.2020-09-28 00:27:31,686 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2020-09-28 00:27:31,687 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-09-28 00:27:31,695 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-09-28 00:27:32,916 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-09-28 00:27:35,999 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-09-28 00:27:36,009 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-09-28 00:27:36,017 sagemake

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [9]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel


# deploy your model to create a predictor
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')


Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


----------------------!CPU times: user 701 ms, sys: 44.1 ms, total: 745 ms
Wall time: 11min 19s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

# Trying with pytorch

In [11]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(

In [12]:
from sagemaker.pytorch import PyTorch

estimatorch = PyTorch(source_dir ='./source_pytorch',
                    entry_point='train.py',
                    role=role,
                    sagemaker_session = sagemaker_session,
                    framework_version='1.5.0',
                    train_instance_count=1,
                    py_version='py3',
                    train_instance_type='ml.m5.xlarge',
                    hyperparameters={
                        'epochs'         : 10,
                        'input_features' : 3,
                        'hidden_dim'     : 64,                        
                        'output_dim'     : 1,     
                    })

In [13]:
%%time 
# train the estimator on S3 training data
estimatorch.fit({'train': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-01 17:39:10 Starting - Starting the training job...
2020-10-01 17:39:12 Starting - Launching requested ML instances......
2020-10-01 17:40:26 Starting - Preparing the instances for training...
2020-10-01 17:41:08 Downloading - Downloading input data...
2020-10-01 17:41:51 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-10-01 17:41:52,475 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-10-01 17:41:52,478 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-10-01 17:41:52,488 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-10-01 17:41:53,100 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-10-01 17:41:53,416 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 


2020-10-01 17:42:09 Uploading - Uploading generated training model
2020-10-01 17:42:09 Completed - Training job completed
Training seconds: 61
Billable seconds: 61
CPU times: user 835 ms, sys: 58 ms, total: 893 ms
Wall time: 3min 31s


In [14]:
from sagemaker.pytorch import PyTorchModel

training_job_name = estimatorch.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
modelTorch = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.5.0',
                     entry_point='predict.py',
                     sagemaker_session=sagemaker_session,
                     source_dir='./source_pytorch')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [15]:
predictorch = modelTorch.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


--------------!

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [16]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [17]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

target_names = ['Non','P']
accuracy = accuracy_score(test_y, test_y_preds)
print('accuracy:',accuracy)
print(classification_report(test_y.values, test_y_preds, target_names=target_names))

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

accuracy: 1.0
              precision    recall  f1-score   support

         Non       1.00      1.00      1.00        10
           P       1.00      1.00      1.00        15

    accuracy                           1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25

1.0

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [18]:
import numpy as np
import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def conf_matrix(real_arr,predict_arr):
    tn, fp, fn, tp = confusion_matrix(real_arr, predict_arr, labels = [0,1]).ravel()
    print("Confusion matrix")
    print("True Positive:",tp)
    print("False Positive:",fp)
    print("False Negative",fn)
    print("True Negative:",tn)
    print(" ")
    print("Metrics")
    print(" ")
    fpr, tpr, thresholds = metrics.roc_curve(real_arr, predict_arr)
    print("Accuracy:",accuracy_score(real_arr, predict_arr))
    print("AUC:", metrics.auc(fpr, tpr))
    print("Precision:",precision_score(real_arr, predict_arr))
    print("Recall:",recall_score(real_arr,predict_arr))
    print("F1-Score:",f1_score(real_arr, predict_arr))
    
conf_matrix(test_y.values, test_y_preds)

Confusion matrix
True Positive: 15
False Positive: 0
False Negative 0
True Negative: 10
 
Metrics
 
Accuracy: 1.0
AUC: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


### Testing pytorch model

In [19]:
# First: generate predicted, class labels
test_y_preds = predictorch.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [20]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

target_names = ['Non','P']
accuracy = accuracy_score(test_y, test_y_preds)
print('accuracy:',accuracy)
print(classification_report(test_y.values, test_y_preds, target_names=target_names))

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

accuracy: 0.6
              precision    recall  f1-score   support

         Non       0.00      0.00      0.00        10
           P       0.60      1.00      0.75        15

    accuracy                           0.60        25
   macro avg       0.30      0.50      0.37        25
weighted avg       0.36      0.60      0.45        25

0.6

Predicted class labels: 
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


/home/robson/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
conf_matrix(test_y.values, test_y_preds)

Confusion matrix
True Positive: 15
False Positive: 10
False Negative 0
True Negative: 0
 
Metrics
 
Accuracy: 0.6
AUC: 0.5
Precision: 0.6
Recall: 1.0
F1-Score: 0.7499999999999999


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

** Answer**: 
False Positive: 0
False Negative 0

There wasn't any kind of wrong prediction.

### Question 2: How did you decide on the type of model to use? 

** Answer**:

I decide to use the Extra Trees Classifier, because it performed better than Pytorch.

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [22]:
# uncomment and fill in the line below!
# <name_of_deployed_predictor>.delete_endpoint()
predictor.delete_endpoint()
predictorch.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [23]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'ABC7B240907F6E43',
   'HostId': '6btXQxdsvbm+xDUarwaRwWsqqnRD9bdSC2OW3O4dKpyC012fgnPRB31ZJ4zl1y2KhLlGY7fnJZI=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '6btXQxdsvbm+xDUarwaRwWsqqnRD9bdSC2OW3O4dKpyC012fgnPRB31ZJ4zl1y2KhLlGY7fnJZI=',
    'x-amz-request-id': 'ABC7B240907F6E43',
    'date': 'Thu, 01 Oct 2020 18:23:22 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'plagiarism_data/train.csv'},
   {'Key': 'pytorch-training-2020-10-01-17-39-08-341/debug-output/events/000000000000/000000000000_worker_0.tfevents'},
   {'Key': 'pytorch-training-2020-10-01-17-39-08-341/output/model.tar.gz'},
   {'Key': 'pytorch-inference-2020-10-01-17-57-18-414/model.tar.gz'},
   {'Key': 'pytorch-training-2020-10-01-17-39-08-341/debug-output/index/000000000/000000000000_worker_0.json'},
   {'Key': 'pytorch-training-2020-10-01-

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!